# Tennis Season Simulation

***Imports***

In [ ]:
"""
Imports that will be used for this assignment
"""

import math
import matplotlib.pyplot as plt
import numpy as np

***Classes***

In [ ]:
class Player():
    # Ensure each new player has a unique ID
    idCounter = 0
    
    def __init__(self, name, serveStrength, returnStrength, form, hardStrength, clayStrength, grassStrength):
        """
        Method:
            init method with adjustable parameters for serve, return strength and form.
        
        Params:
            name (str):               The name of the player
            serveStrength (float):    A multiplier for the players serve effectiveness
            returnStrength (float):   A multiplier for the players return effectiveness
            form (float):             A multiplier for the players current form
            hardStrength (float):     A multiplier for the players strength on hard courts
            clayStrength (float):     A multiplier for the players strength on clay courts
            grassStrength (float):    A multiplier for the players strength on grass courts
        """
        # Player identity
        self.name = name
        self.playerID = Player.idCounter
                
        # Player attributes
        self.serveStrength = 58*serveStrength # Tour average of service points won is 58%
        self.returnStrength = 42*returnStrength # Tour average of return points won is 42%
        self.form = form
        self.hardStrength = hardStrength
        self.clayStrength = clayStrength
        self.grassStrength = grassStrength

        # Player Stats
        self.gamesPlayer = 0
        self.tournamentsWon = {
            "Grand Slam": 0,
            "ATP Final": 0,
            "Master 1000": 0,
            "ATP 500": 0,
            "ATP 250": 0
        }

        # Increment ID counter to ensure every player has a unique ID
        Player.idCounter += 1
        
    def __init__(self, name):
        """
        Method:
            init method where serve, return and form multipliers are assigned random values
        
        Params:
            name (str): The name of the player
        """
        # Player Identity
        self.name = name
        self.playerID = Player.idCounter
        
        # Player attributes
        self.serveStrength = 58*np.random.normal(loc=1.0, scale=0.5) # Tour average of service points won is 58%
        self.returnStrength = 42*np.random.normal(loc=1.0, scale=0.5)  # Tour average of return points won is 42%
        self.form = np.random.normal(loc=1.0, scale=1)
        self.hardStrength = np.random.normal(loc=1.0, scale=1)
        self.clayStrength = np.random.normal(loc=1.0, scale=1)
        self.grassStrength = np.random.normal(loc=1.0, scale=1)
        
        # Player stats
        self.gamesPlayed = 0
        self.tournamentsWon = {
            "Grand Slam": 0,
            "ATP Final": 0,
            "Master 1000": 0,
            "ATP 500": 0,
            "ATP 250": 0
        }
        self.rankingPoints = 0
        
        # Increment ID counter to ensure every player has a unique ID
        Player.idCounter += 1
        
    def increment_tournament_win(self, tournamentType):
        """
        Method:
            Increments the type of tournament the player won
            
        Params:
            tournamentType (str): The type of tournament won e.g. ATP 500
        """
        self.tournamentsWon[tournamentType] += 1
    
    def increment_points(self, points):
        """
        Method:
            Increments the players ranking points by given points
            
        Params:
            points (int): The amount of points to increment by
        """
        self.rankingPoints += points
        
    def increment_games_played(self):
        """
        Method:
            Increments the gamesPlayed attribute
        """
        self.gamesPlayed += 1

In [4]:
class Game():
    def __init__(self, server, returner):
        """ 
        Method:
            init method for Game class
            
        Params:
            server (Player):   The player that is serving in the game
            returner (Player): The player that is returning in the game
        """
        # Players
        self.server = server
        self.returner = returner
        
        # Markov Chain
        self.markovChainIndex = self.generate_markov_chain_index()
        self.markovChain = self.generate_markov_chain()
        
        # Points
        self.score = (0,0)
        self.pointProgression = [0, 15, 30, 40, ("Hold", "Break")]
        self.winner = None
    
    def simulate_point(self):
        """ 
        Method:
            Simulates a point between the two players based off of
            probabilities obtained from the Markov Chain
            
        Return:
            updatedScore (tuple(int|str,int|str)): The updated score after point simulation
        """
        # Fetching each players points
        serverPoints = self.score[0]
        returnerPoints = self.score[1]
        
        # Fetching target scores for both players
        serverTargetIndex = self.pointProgression.index(serverPoints) + 1
        returnerTargetIndex = self.pointProgression.index(returnerPoints) + 1
        serverTargetScore = self.pointProgression[serverTargetIndex]
        returnerTargetScore = self.pointProgression[returnerTargetIndex]
        
        # Checking whether the target score results in a hold or a break
        if serverTargetIndex == len(self.pointProgression) - 1:
            serverTargetScore = "Hold"
        elif returnerTargetIndex == len(self.pointProgression) - 1:
            returnerTargetScore = "Break"
        
        # Fetching probability that server wins the point
        currentMarkovIndex = self.markovChainIndex[self.format_score(serverPoints,returnerPoints)]
        targetMarkovIndex = self.markovChainIndex[self.format_score(serverTargetScore, returnerPoints)]
        serverProb = self.markovChainIndex[currentMarkovIndex, targetMarkovIndex]
        
        # Randomly sampling to see if server wins the point
        sample = np.random.uniform(low=0,high=1)
        
        # Updating score based on random sample
        if sample <= serverProb:
            updatedScore = (serverTargetScore, returnerPoints)
        else:
            updatedScore = (serverPoints, returnerTargetScore)
        
        # If the updated score results in deuce we need to set it to 30-30 so markov indexing works
        if updatedScore == (40,40):
            updatedScore = (30,30)
        
        return updatedScore
            
    def simulate_game(self):
        """ 
        Method:
            Simulates the game
        """
        while self.winner == None:
            # Simulate the point
            updatedScore = self.simulate_point()
            
            # If the game has a winner update to exit loop 
            if "Hold" in updatedScore:
                self.winner = self.server
            elif "Break" in updatedScore:
                self.winner = self.returner
            
            # Update score
            self.score = updatedScore
            
    
    def generate_markov_chain(self):
        """
        Method:
            Generates a markov chain of the point transitions of the game.
            Calculates probability of server winning the point based off of both
            the server and returners personal attributes. For simplicity deuce is 
            simplified to 30-30 as they are essentially the same thing.
        
        Return:
            markovChain (List(List(float))): The generated markov chain
        """
        # Obtaining probability that server will win each point based on player attributes
        sWin = (self.server.serveStrength * self.server.form)/(self.server.serveStrength * self.server.form + self.returner.returnStrength * self.returner.form)
        
        # The probability that the returner will win the point is just the compliment of the server's probability
        rWin = 1 - sWin
        
        # Creating Markov chain
        markovChain = np.array([# 0-0   15-0    30-0    40-0    0-15    0-30    0-40    15-15   30-15   40-15   15-30   15-40   30-30   40-30   30-40   Hold    Break
                                [  0,   sWin,     0,      0,    rWin,     0,      0,      0,      0,      0,      0,       0,     0,      0,      0,     0,      0    ],# 0-0     
                                [  0,     0,    sWin,     0,      0,      0,      0,     rWin,    0,      0,      0,       0,     0,      0,      0,     0,      0    ],# 15-0
                                [  0,     0,      0,    sWin,     0,      0,      0,      0,     rWin,    0,      0,       0,     0,      0,      0,     0,      0    ],# 30-0
                                [  0,     0,      0,      0,      0,      0,      0,      0,      0,    rWin,     0,       0,     0,      0,      0,    sWin,    0    ],# 40-0      
                                [  0,     0,      0,      0,      0,    rWin,     0,     sWin,    0,      0,      0,       0,     0,      0,      0,     0,      0    ],# 0-15  
                                [  0,     0,      0,      0,      0,      0,      rWin,   0,      0,      0,     sWin,     0,     0,      0,      0,     0,      0    ],# 0-30
                                [  0,     0,      0,      0,      0,      0,      0,      0,      0,      0,      0,     sWin,    0,      0,      0,     0,      rWin ],# 0-40
                                [  0,     0,      0,      0,      0,      0,      0,      0,     sWin,    0,     rWin,     0,     0,      0,      0,     0,      0    ],# 15-15
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     sWin,   0,       0,    rWin,    0,      0,     0,      0    ],# 30-15
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     0,     rWin,    0,     sWin,   0    ],# 40-15
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,     rWin,   sWin,    0,      0,     0,      0    ],# 15-30
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     0,      0,     sWin,   0,     rWin  ],# 15-40
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     0,      sWin,  rWin,   0,      0    ],# 30-30
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     rWin,   0,      0,     sWin,   0    ],# 40-30 (A-40)  
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     sWin,   0,      0,     0,     rWin  ],# 30-40 (40-A)
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     0,      0,      0,     1,      0    ],# Hold
                                [  0,     0,      0,      0,      0,      0,      0,      0,       0,     0,      0,       0,     0,      0,      0,     0,      1    ] # Break
                                ])
        
        return markovChain
    
    def generate_markov_chain_index(self):
        """ 
        Method:
            Returns a dictionary of the indexes for the markov
            chain given a score
        
        Return:
            markovIndexes (dict)
        """
        return { "0-0" : 0, "15-0" : 1, "30-0" : 2, "40-0" : 3, "0-15" : 4, "0-30" : 5, "0-40" : 6, "15-15" : 7, "30-15" : 8, "40-15" : 9, "15-30" : 10, "15-40" : 11, "30-30" : 12, "40-30" : 13, "30-40" : 14, "Hold" :15, "Break" : 16 }
    
    def format_score(self, score=None):
        """ 
        Method:
            Formats a score to a string
        
        Params:
            score (None|tuple(int|str, int|str)): The score that will be formatted. If none,
                                                  formats the game classes own score.
                        
        Return:
            formattedScore (str): The formatted score
        """
        if score != None:
            return f'{score[0]}-{score[1]}'
        else:
            return f'{self.score[0]}-{self.score[1]}'

In [ ]:
class Set():
    pass

In [ ]:
class Match():
    pass

In [3]:
class Tournament():
    def __init__(self, courtType):
        self.courtType = courtType
        
    def get_points_for_round(self, round):
        """
        Method:
            Returns the points associated for a given round
            
        Params:
            round (str): The given round
        
        Returns:
            roundPoints (int): The points associated with the round
        """
        return self.roundPoints[round]
    
    def generate_draw(self, players):
        """
        Method:
            Generates the draw for a tournament 
        
        Params:
            players (List(Player)): The list of the players playing the tournament
            
        Returns:
            draw (List(Match)): A list that contains all the first round matches
         
        TODO: Create draw so that seeded pairings 1,2,3,4 are all on opposite sides
        """
        # Sort players in terms of rankings
        players = seed(players)
        
        # Seperate seeded players and non seeded players
        seededPlayers = players[:(self.drawsize/4)]
        nonSeededPlayers = players[(self.drawsize/4):]
        
        # Initial draw variables
        draw = []
        
        # Getting first matches for round 1 based off of seedings
        for match in range(self.drawsize/2):
            # getting random players that are seeded and not seeded
            if len(seededPlayers) > 0:
                seededIndex = int(np.random.uniform(low=0,high=len(seededPlayers)))
                nonSeededIndex = int(np.random.uniform(low=0,high=len(nonSeededPlayers)))
            # Some matches will have two non seeded players playing eachother
            else:
                seededIndex = int(np.random.uniform(low=0,high=len(nonSeededPlayers)))
                nonSeededIndex = int(np.random.uniform(low=0,high=len(nonSeededPlayers)))
            
            # Appending them to the first round
            draw.append(Match(seededPlayers[seededIndex], nonSeededPlayers[nonSeededIndex]))
            
            # Deleting random players from their respective lists
            del seededPlayers[seededIndex]
            del nonSeededPlayers[nonSeededIndex]
            
        return draw
        
        
class GrandSlam(Tournament):
    def __init__(self, courtType):
        super.__init__(self, courtType)
        self.roundPoints = {
            "R1": 10,
            "R2": 45,
            "R3": 90,
            "R4": 180,
            "quarter final": 360,
            "semi final": 720,
            "finalist": 1200,
            "winner": 2000
        }
        self.drawSize = 128

class Master1000(Tournament):
    def __init__(self, courtType):
        super.__init__(self, courtType)
        self.roundPoints = {
            "R1": 10,
            "R2": 25,
            "R3": 45,
            "R4": 90,
            "quarter final": 180,
            "semi final": 360,
            "finalist": 600,
            "winner": 1000
        }
        self.drawSize = 128 

class ATP500(Tournament):
    def __init__(self, courtType):
        super.__init__(self, courtType)
        self.roundPoints = {
            "R1": 20,
            "R2": 45,
            "quarter final": 90,
            "semi final": 180,
            "finalist": 300,
            "winner": 500
        }
        self.drawSize = 64

class ATP250(Tournament):
    def __init__(self, courtType):
        super.__init__(self, courtType)
        self.roundPoints = {
            "R1": 10,
            "R2": 20,
            "quarter final": 45,
            "semi final": 90,
            "finalist": 150,
            "winner": 250
        }
        self.drawSize = 64


In [1]:
class Season():
    pass

***Main***